# Sharrow Basics

This notebook provides a short walkthrough of some of the basic features of the `sharrow` library.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from io import StringIO

import sharrow as sh
sh.__version__

In [ ]:
import sys
sys.version_info < (3,8)

In [ ]:
# check versions
import packaging
assert packaging.version.parse(xr.__version__) >= packaging.version.parse("0.20.2")

## Example Data

We'll begin by importing some example data to work with.  We'll be using 
some test data taken from the MTC example in the ActivitySim project, including 
tables of data for households and persons, as well as a set of 
skims containing transportation level of service information for travel around
a tiny slice of San Francisco.

The households and persons are typical tabular data, and 
each can be read in and stored as a `pandas.DataFrame`.

In [ ]:
households = sh.example_data.get_households()
households.head()

In [ ]:
# test households content
assert len(households) == 5000
assert "income" in households 
assert households.index.name == "HHID"

In [ ]:
persons = sh.example_data.get_persons()
persons.head()

In [ ]:
assert len(persons) == 8212
assert "household_id" in persons
assert persons.index.name == 'PERID'

The skims, on the other hand, are not just simple tabular data, but rather a 
multi-dimensional representation of the transportation system, indexed by origin.
destination, and time of day. Rather than using a single DataFrame for this data,
we store it as a multi-dimensional `xarray.Dataset`.

In [ ]:
skims = sh.example_data.get_skims()
skims

For tabular data, sharrow can be provided either pandas DataFrames or xarray Datasets, 
but to ensure consistency the former are converted into the latter automatically when
they are used with sharrow.  You can also easily manually make the conversion:

In [ ]:
xr.Dataset(persons)

Suppose we're wanting to simulate a tour mode choice.  Normally we'd probably have
run through a bunch of different models to generate these tours and their destinations
first, but let's just skip that for now and make up some random data to work with.  We'll 
just randomly choose (with replacement) 100,000 people, and send them to 100,000 zones, with
random outbound and inbound time periods.

In [ ]:
def random_tours(n_tours=100_000, seed=42):
    rng = np.random.default_rng(seed)
    n_zones = skims.dims['dtaz']
    return pd.DataFrame({
        'PERID': rng.choice(persons.index, size=n_tours),
        'dest_taz_idx': rng.choice(n_zones, size=n_tours),
        'out_time_period': rng.choice(skims.time_period, size=n_tours),
        'in_time_period': rng.choice(skims.time_period, size=n_tours),
    }).rename_axis("TOURIDX")
tours = random_tours()
tours.head()

In [ ]:
assert tours.index.name == "TOURIDX"
assert 0 in tours.head().dest_taz_idx

Of note in this table, we include include destination TAZ's by index (position) not 
label, so we can observe a TAZ index of `0` even though the first TAZ ID is 1.

## Spec Files

Now that we've got our tours to work with, we'll also need 
an expression "spec" file that defines the utility function
terms and coefficients.  Following the ActivitySim format, we
can write a mini-spec file as appears below.  Each line of this
CSV file has an expression that can be evaluated in the context
of the various tables and datasets shown above, plus a set of 
coefficients that apply for that expression across various modal 
alternatives (drive, walk, and transit in this example).

In [ ]:
mini_spec = """
Label,Expression,DRIVE,WALK,TRANSIT
Drive Time,odt_skims['SOV_TIME'] + dot_skims['SOV_TIME'],-0.0134,,
Transit IVT,(odt_skims['WLK_LOC_WLK_TOTIVT']/100 + dot_skims['WLK_LOC_WLK_TOTIVT']/100),,,-0.0134
Transit Wait Time,short_i_wait_mult * ((odt_skims['WLK_LOC_WLK_IWAIT']/100).clip(upper=shortwait) + (dot_skims['WLK_LOC_WLK_IWAIT']/100).clip(upper=shortwait)),,,-0.0134
Income,hh.income > 60000,,-0.2,
Constant,1,,-0.4,-0.55
"""

We'll use pandas to load these values into a DataFrame.

In [ ]:
spec = pd.read_csv(StringIO(mini_spec), index_col='Label')
spec

In [ ]:
assert spec.index.name == "Label"
assert all(spec.columns == ['Expression', 'DRIVE', 'WALK', 'TRANSIT'])

## Data Trees and Flows

Then, it's time to prepare our data.  We'll create a `DataTree`
that defines the relationships among all the datasets we're working
with.  This is a tree in the mathematical sense, with nodes referencing
the datasets and edges representing the relationships.

In [ ]:
tree = sh.DataTree(
    tour=tours,
    person=persons,
    hh=households,
    odt_skims=skims,
    dot_skims=skims,
    relationships=(
        "tour.PERID @ person.PERID",
        "person.household_id @ hh.HHID",
        "hh.TAZ @ odt_skims.otaz",
        "tour.dest_taz_idx -> odt_skims.dtaz",
        "tour.out_time_period @ odt_skims.time_period",
        "tour.dest_taz_idx -> dot_skims.otaz",
        "hh.TAZ @ dot_skims.dtaz",
        "tour.in_time_period @ dot_skims.time_period",
    ),
    extra_vars={
        'short_i_wait_mult': 0.75,
        'shortwait': 3,
    },
)

The first named dataset we include, `tour`, is by default the root node of this data tree.
We then can define an arbitrary number of other named data nodes.  Here, we add `person`, `hh`,
`odt_skims` and `odt_skims`.  Note that these last two are actually two different names for the
same underlying dataset, and for each name we will next define a unique set of relationships.

All data nodes in this tree are stored as `Dataset` objects. We can give a pandas DataFrame
in this contructor instead, but it will be automatically converted into a one-dimension `Dataset`.
The conversion is no-copy if possible (and it is usually possible) so no additional memory is
consumed in the conversion.

The `relationships` defines links of the data tree. Each relationship maps a particular variable
in a named upstream dataset to a particular dimension of a named downstream dataset.  For example,
`"person.household_id @ hh.HHID"` tells the tree that the `household_id` variable in the `person` 
dataset contains labels (`@`) that map to the `HHID` dimension of the `hh` dataset.

In addition to mapping by label, we can also map by position, by using the `->` operator in the
relationship string instead of `@`.  In the example above, we map the tour destination TAZ's in
this manner, as the `dest_taz_idx` variable in the `tours` dataset contains positional references
instead of labels.

Lastly, out tree definition includes a few named constants, that are just fixed values defined
in a separate dictionary. These values get hard-coded into the compiled results, effectively the 
same as if their values were expanded and written into exprssions in the `spec` directly.

Once we have defined our data tree, we can use it along with the `spec`, to compute the utility
for various alternatives in the choice model.  Sharrow allows us to compile this utility function
into a `Flow`, which can be reused for massive speed gains on later utility evaluations.

In [ ]:
flow = tree.setup_flow(spec.Expression)

To use a `Flow` for preparing the array of data that backs the utility
function, we can call the `load()` method. The first time we call `load()`,
it takes a (relatively) long time to evaluate, as the expressions are compiled
and that compiled code is cached to disk.

In [ ]:
%time flow.load(compile_watch=True)

In [ ]:
# test utility data
assert flow.compiled_recently == True
actual = flow.load()
expected = np.array([[  9.4     ,  16.9572  ,   4.5     ,   0.      ,   1.      ],
       [  9.32    ,  14.3628  ,   4.5     ,   1.      ,   1.      ],
       [  7.62    ,  11.0129  ,   4.5     ,   1.      ,   1.      ],
       [  4.25    ,   7.6692  ,   2.50065 ,   0.      ,   1.      ],
       [  6.16    ,   8.2186  ,   3.387825,   0.      ,   1.      ],
       [  4.86    ,   4.9288  ,   4.5     ,   0.      ,   1.      ],
       [  1.07    ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  8.52    ,  11.615499,   3.260325,   0.      ,   1.      ],
       [ 11.74    ,  16.2798  ,   3.440325,   0.      ,   1.      ],
       [ 10.48    ,  13.3974  ,   3.942825,   0.      ,   1.      ]], dtype=np.float32)

np.testing.assert_array_almost_equal(actual[:5], expected[:5])
np.testing.assert_array_almost_equal(actual[-5:], expected[-5:])
assert actual.shape == (len(tours), len(spec))

Subsequent calls to `load()` are much faster.

In [ ]:
%time flow.load(compile_watch=True)

In [ ]:
# test compile flags
assert flow.compiled_recently == False
flow.load(compile_watch=False)
import pytest
with pytest.raises(AttributeError):
    flow.compiled_recently # attribute does not exist if compile_watch flag is off

It's not faster because it's cached the data, but because it's cached the compiled code.
(Setting the `compile_watch` argument to a truthy value will trigger a check of the 
cache files and emit a warning message if recompilation was triggered.)
We can swap out the `tour` node in the tree for a different set of (similarly formatted)
tours, and re-evaluate at that fast speed.

In [ ]:
tours_2 = random_tours(seed=43)
tours_2.head()

Note that the flow requires not just a base dataset but a whole DataTree to operate,
so to re-evaluate with a new `tours` we need to make a DataTree with `replace_datasets`.
Fortuntately, this operation is no-copy so it doesn't consume much memory.  If all the 
datasets in a tree are linked by position (instead of by label) this would be almost 
instantaneous, but since our example tree here has tours linked by label it takes just a
moment to rebuild the linkages.

In [ ]:
tree_2 = tree.replace_datasets(tour=tours_2)

In [ ]:
%time flow.load(tree_2)

The load function also has some other features, like nicely formatting the output
into a DataFrame.

In [ ]:
df = flow.load_dataframe()
df

In [ ]:
# test df
assert len(df) == len(tours)
pd.testing.assert_index_equal(
    df.columns, 
    pd.Index(['Drive Time', 'Transit IVT', 'Transit Wait Time', 'Income', 'Constant']),
)
expected_df_head = pd.read_csv(StringIO(''',Drive Time,Transit IVT,Transit Wait Time,Income,Constant
0,9.4,16.9572,4.5,0.0,1.0
1,9.32,14.3628,4.5,1.0,1.0
2,7.62,11.0129,4.5,1.0,1.0
3,4.25,7.6692,2.50065,0.0,1.0
4,6.16,8.2186,3.387825,0.0,1.0'''), index_col=0).astype(np.float32)
pd.testing.assert_frame_equal(df.head(), expected_df_head)

## Linear-in-Parameters Functions

When the `spec` represents a linear-in-parameters utility function, the data 
we get out of the `load()` function represents one matrix in a dot-product, and
the coefficients in the `spec` provide the other matrix.  We might look to 
use the efficient linear algebra algorithms embedded in `np.dot` to compute the
utility, like this:

In [ ]:
x = flow.load()
b = spec.iloc[:,1:].fillna(0).astype(np.float32).values
np.dot(x, b)

But `sharrow` provides a substantially faster option, by embedding
the dot product directly into the compiled code and never instantiating the
full `x` array in memory at all.

In [ ]:
%time u = flow.dot(b)
u

In [ ]:
# test utility
np.testing.assert_array_almost_equal(u, np.dot(x, b))

As before, the compiler runs only the first time we apply the this 
function with this structure, and subsequent runs are faster, even with
different source data.

In [ ]:
%time flow.dot(b, source=tree_2)

As for the plain `load` method, the `dot` method also has some formatted output versions.
For example, the `dot_dataarray` returns a `DataArray`.

In [ ]:
flow.dot_dataarray(b, source=tree_2)

This works even better if the coefficients are given as a DataArray too, so it 
can harvest dimension names and coordinates as appropriate.

In [ ]:
B = xr.DataArray(
    spec.iloc[:,1:].fillna(0).astype(np.float32), 
    dims=('expressions','modes')
)
flow.dot_dataarray(B, source=tree_2)

## Multinomial Logit Simulation

The next level of flow evaluation is made by treating the dot-product as a
linear-in-parameters multinomial logit (MNL) utility function, and making simulated
choices based on that model.  To do this, we'll need to provide the random
draws as a function input (which also lets us attach any randomization engine
we prefer, e.g. a reproducible random generator).  For this example, we'll 
create one random (uniform) draw for each tour.

In [ ]:
draws = np.random.default_rng(321).random(size=tree.shape[0])

Given those draws, we use the `mnl_draws` method to build and apply a 
MNL simulator, which returns to us both the choices and the probability that
was computed for each chosen alternative. 

In [ ]:
%time choices, choice_probs = flow.mnl_draws(b, draws)

As this is the most complex flow processor,
it takes the longest to compile, but after compilation it runs quite efficiently.
We can see here the whole MNL simulation process for this data requires only a few 
milliseconds more time than just computing the utilities.

In [ ]:
%time choices2, choice_probs2 = flow.mnl_draws(b, draws, source=tree_2)

The resulting choices are the index position of the choices, not the labels.

In [ ]:
choices

But if we want the labels, it's easy enough to convert these indexes into labels.

In [ ]:
B.modes[choices]

In [ ]:
# test mnl choices
uz = np.exp(flow.dot(b))
uz = uz / uz.sum(1)[:,None]
np.testing.assert_array_almost_equal(
    uz[range(uz.shape[0]),choices.ravel()],
    choice_probs.ravel(),
)